In [3]:
!pip install pymysql==0.9.3
!pip install mysql.connector

Processing /root/.cache/pip/wheels/8c/83/a1/f8b6d4bb1bd6208bbde1608bbfa7557504bed9eaf2ecf8c175/mysql_connector-2.2.9-cp37-cp37m-linux_x86_64.whl


In [1]:
import io
import pandas as pd
from google.colab import files
uploaded = files.upload()

Saving trips_new.csv to trips_new (2).csv


In [37]:
trips = pd.read_csv("trips_new.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [38]:
trips = trips.dropna().reset_index()
trips

,index,date,state,fips,pop_home,pop_not_home,trips,trips1,trips1to3,trips3to5,trips5to10,trips10to25,trips25to50,trips50to100,trips100to250,trips500
0,0,1/1/2019,AK,200344.0,534838.0,1942538.0,558829.0,529269.0,242666.0,264106.0,203362.0,87492.0,34583.0,7591.0,5765.0,8875.0
1,1,1/1/2019,FL,4524666.0,16709556.0,55258300.0,16036792.0,14469767.0,6658624.0,7977612.0,6604304.0,2113009.0,796740.0,412224.0,95341.0,93887.0
2,2,1/1/2019,DC,241030.0,459278.0,3114055.0,1314825.0,833796.0,367547.0,344116.0,183744.0,42096.0,10730.0,8109.0,3476.0,5616.0
3,3,1/1/2019,DE,224457.0,739757.0,2420942.0,584323.0,640348.0,305441.0,370359.0,314102.0,128454.0,50542.0,22701.0,2474.0,2198.0
4,4,1/1/2019,CT,853894.0,2707851.0,9468961.0,2375021.0,2618933.0,1189575.0,1331356.0,1211252.0,449947.0,192960.0,56618.0,12031.0,31268.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40948,40948,13/3/2021,CO,1740765.0,4017971.0,19858148.0,5660870.0,5551776.0,2622873.0,2926542.0,2226670.0,526796.0,192922.0,89645.0,29739.0,30315.0
40949,40949,13/3/2021,CA,11537825.0,27974398.0,104905625.0,29787973.0,26122282.0,12483876.0,14790491.0,13345582.0,5290202.0,2157157.0,725676.0,133779.0,68607.0
40950,40950,13/3/2021,AR,491958.0,2525846.0,11302022.0,2611323.0,2665713.0,1411261.0,1783741.0,1712046.0,692718.0,321201.0,92926.0,9154.0,1939.0
40951,40951,13/3/2021,AZ,1613936.0,5664781.0,25399486.0,6902255.0,5901237.0,3111741.0,3924964.0,3714566.0,1143859.0,416727.0,231437.0,28503.0,24197.0


In [39]:
trips["date"] = trips["date"].apply(lambda x: x.replace("/", "-"))
trips = trips.reset_index()
trips = trips.drop(columns=['index', 'level_0'])

In [40]:
trips

,date,state,fips,pop_home,pop_not_home,trips,trips1,trips1to3,trips3to5,trips5to10,trips10to25,trips25to50,trips50to100,trips100to250,trips500
0,1-1-2019,AK,200344.0,534838.0,1942538.0,558829.0,529269.0,242666.0,264106.0,203362.0,87492.0,34583.0,7591.0,5765.0,8875.0
1,1-1-2019,FL,4524666.0,16709556.0,55258300.0,16036792.0,14469767.0,6658624.0,7977612.0,6604304.0,2113009.0,796740.0,412224.0,95341.0,93887.0
2,1-1-2019,DC,241030.0,459278.0,3114055.0,1314825.0,833796.0,367547.0,344116.0,183744.0,42096.0,10730.0,8109.0,3476.0,5616.0
3,1-1-2019,DE,224457.0,739757.0,2420942.0,584323.0,640348.0,305441.0,370359.0,314102.0,128454.0,50542.0,22701.0,2474.0,2198.0
4,1-1-2019,CT,853894.0,2707851.0,9468961.0,2375021.0,2618933.0,1189575.0,1331356.0,1211252.0,449947.0,192960.0,56618.0,12031.0,31268.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40948,13-3-2021,CO,1740765.0,4017971.0,19858148.0,5660870.0,5551776.0,2622873.0,2926542.0,2226670.0,526796.0,192922.0,89645.0,29739.0,30315.0
40949,13-3-2021,CA,11537825.0,27974398.0,104905625.0,29787973.0,26122282.0,12483876.0,14790491.0,13345582.0,5290202.0,2157157.0,725676.0,133779.0,68607.0
40950,13-3-2021,AR,491958.0,2525846.0,11302022.0,2611323.0,2665713.0,1411261.0,1783741.0,1712046.0,692718.0,321201.0,92926.0,9154.0,1939.0
40951,13-3-2021,AZ,1613936.0,5664781.0,25399486.0,6902255.0,5901237.0,3111741.0,3924964.0,3714566.0,1143859.0,416727.0,231437.0,28503.0,24197.0


In [46]:
tripper = pd.DataFrame()

tripper['pop_home'] = trips.groupby(['date'])['pop_home'].mean()
tripper['pop_not_home'] = trips.groupby(['date'])['pop_not_home'].mean()
tripper['trips'] = trips.groupby(['date'])['trips'].sum()
tripper['trips1'] = trips.groupby(['date'])['trips1'].sum()
tripper['trips1to3'] = trips.groupby(['date'])['trips1to3'].sum()
tripper['trips3to5'] = trips.groupby(['date'])['trips3to5'].sum()
tripper['trips5to10'] = trips.groupby(['date'])['trips5to10'].sum()
tripper['trips10to25'] = trips.groupby(['date'])['trips10to25'].sum()
tripper['trips25to50'] = trips.groupby(['date'])['trips25to50'].sum()
tripper['trips50to100'] = trips.groupby(['date'])['trips50to100'].sum()
tripper['trips100to250'] = trips.groupby(['date'])['trips100to250'].sum()
tripper['trips500'] = trips.groupby(['date'])['trips500'].sum()
tripper['date'] = tripper.index

tripper["pop_home"] = pd.to_numeric(tripper["pop_home"], downcast='signed')
tripper["pop_not_home"] = pd.to_numeric(tripper["pop_not_home"], downcast='signed')

tripper = tripper.tail(-5)

In [47]:
tripper

,pop_home,pop_not_home,trips,trips1,trips1to3,trips3to5,trips5to10,trips10to25,trips25to50,trips50to100,trips100to250,trips500,date
date,,,,,,,,,,,,,
21-1-2019,5.146907e+06,2.187103e+07,282498500.0,285772483.0,137908784.0,167618586.0,157962186.0,53770892.0,18330939.0,8166452.0,1919540.0,1474124.0,21-1-2019
21-1-2020,5.151149e+06,2.347572e+07,305811768.0,296486824.0,144464973.0,182155315.0,181648239.0,60712642.0,17713419.0,6194349.0,1315925.0,758222.0,21-1-2020
21-1-2021,4.871800e+06,2.079669e+07,275650114.0,255792671.0,130546724.0,165703732.0,159783150.0,50230983.0,14766763.0,5895182.0,1381443.0,880649.0,21-1-2021
21-10-2019,5.101097e+06,2.563402e+07,323459054.0,310781978.0,158943880.0,208851951.0,218992582.0,64721180.0,15227545.0,4317755.0,971498.0,1067407.0,21-10-2019
21-10-2020,4.730101e+06,1.746092e+07,200050742.0,218581270.0,106861139.0,137161314.0,142209139.0,54390616.0,21070218.0,8080402.0,1547071.0,555010.0,21-10-2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9-7-2020,4.893847e+06,1.867305e+07,250109844.0,224635410.0,110408483.0,140786478.0,141783496.0,51526652.0,19927368.0,9805768.0,2196786.0,1145394.0,9-7-2020
9-8-2019,5.335322e+06,2.944259e+07,341515854.0,378936486.0,186257669.0,242027669.0,245571995.0,74512584.0,20493069.0,7635743.0,1823661.0,2797133.0,9-8-2019
9-8-2020,4.742312e+06,1.778921e+07,220836790.0,230115797.0,110985230.0,135758155.0,127217608.0,46082141.0,21302378.0,11664667.0,2366289.0,920610.0,9-8-2020


In [63]:
tripper = tripper.set_index(pd.to_datetime(tripper['date']))
tripper = tripper[(tripper['date'] >= '2019-12-22')]
tripper = tripper.resample('D').interpolate()[::7]
tripper['date'] = tripper.index
tripper['date'] = tripper['date'].astype(str)
tripper

,pop_home,pop_not_home,trips,trips1,trips1to3,trips3to5,trips5to10,trips10to25,trips25to50,trips50to100,trips100to250,trips500,date
date,,,,,,,,,,,,,
2019-12-23,5.155950e+06,2.724014e+07,3.272320e+08,3.331928e+08,1.758627e+08,2.272009e+08,2.302487e+08,6.807656e+07,1.802214e+07,6.197282e+06,1.481051e+06,1.733338e+06,2019-12-23
2019-12-30,5.100384e+06,2.202755e+07,2.350972e+08,2.902410e+08,1.526289e+08,1.909603e+08,1.837683e+08,5.176679e+07,1.291124e+07,4.368809e+06,9.032170e+05,7.590510e+05,2019-12-30
2020-01-06,5.252801e+06,2.755379e+07,3.207608e+08,3.519464e+08,1.827175e+08,2.305069e+08,2.278808e+08,6.745272e+07,1.696670e+07,5.267778e+06,1.039691e+06,7.042241e+05,2020-01-06
2020-01-13,5.205364e+06,2.565069e+07,3.137846e+08,3.260653e+08,1.648663e+08,2.079428e+08,2.063056e+08,6.430735e+07,1.731517e+07,5.700178e+06,1.168600e+06,7.294231e+05,2020-01-13
2020-01-20,5.157926e+06,2.374759e+07,3.068084e+08,3.001841e+08,1.470151e+08,1.853788e+08,1.847304e+08,6.116198e+07,1.766364e+07,6.132578e+06,1.297509e+06,7.546221e+05,2020-01-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-02,4.828635e+06,2.245206e+07,3.039904e+08,2.654015e+08,1.376978e+08,1.775052e+08,1.738057e+08,5.661225e+07,1.869138e+07,8.350681e+06,1.786270e+06,1.213868e+06,2021-08-02
2021-08-09,4.954493e+06,2.309864e+07,3.160562e+08,2.830767e+08,1.436531e+08,1.814970e+08,1.714115e+08,5.477910e+07,1.749956e+07,7.465945e+06,1.572040e+06,1.019663e+06,2021-08-09
2021-08-16,4.886156e+06,2.203850e+07,2.984280e+08,2.736706e+08,1.380436e+08,1.724966e+08,1.609776e+08,5.200726e+07,1.741085e+07,8.275647e+06,1.684909e+06,9.682419e+05,2021-08-16


In [64]:
import pandas as pd
import pymysql
import mysql.connector
import json,gzip, pandas as pd
from sqlalchemy import create_engine
from tqdm import tqdm

In [69]:
host="147.8.122.231"
port=3306
dbname="writeable_fina_4359_u3555664"
user="fina_4359_u3555664"
password="cb485"

con = pymysql.connect(host, port = 3306, user= user, passwd = password, db = dbname)

In [70]:
cnx_str = ('mysql+pymysql://{username}:{password}@{ipaddress}:{port}/{dbname}'
.format(username=user,
password=password,
ipaddress=host,
port=port,
dbname=dbname))

cnx = create_engine(cnx_str, echo=True)

In [71]:
try:
  with con.cursor() as cur:
    cur.execute('SHOW DATABASES')
    row = cur.fetchall()
    for rows in row:
      print(rows)
    cur.connection.commit()
finally:
  pass

('class_demo',)
('comp',)
('crsp',)
('ff',)
('information_schema',)
('instown',)
('link',)
('temp',)
('writeable_fina_4359_u3548379',)


In [72]:
#cnx.execute("CREATE TABLE visits (date DATE, postal VARCHAR(255), fips INT(255), home INT(255),not_home INT(255), trips INT(255), trips1 INT(255), trips3 INT(255), trips5 INT(255), trips10 INT(255), trips25 INT(255), trips50 INT(255), trips100 INT(255), trips250 INT(255), trips500 INT(255), tripsPlus INT(255) )")

In [73]:
tripper.to_sql(con=cnx, name = "visits", if_exists="replace", index=False)

2021-05-25 13:45:15,651 INFO sqlalchemy.engine.Engine SHOW VARIABLES LIKE 'sql_mode'
2021-05-25 13:45:15,652 INFO sqlalchemy.engine.Engine [raw sql] {}
2021-05-25 13:45:15,830 INFO sqlalchemy.engine.Engine SHOW VARIABLES LIKE 'lower_case_table_names'
2021-05-25 13:45:15,832 INFO sqlalchemy.engine.Engine [generated in 0.00205s] {}
2021-05-25 13:45:16,176 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2021-05-25 13:45:16,178 INFO sqlalchemy.engine.Engine [raw sql] {}
2021-05-25 13:45:16,690 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2021-05-25 13:45:16,691 INFO sqlalchemy.engine.Engine [generated in 0.00166s] {'table_schema': 'writeable_fina_4359_u3548379', 'table_name': 'visits'}
2021-05-25 13:45:17,035 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2021-05-25 13:45:17,038 INFO sqlalch